# Part 2: Spatial Interaction models
For this section, you will be given a “symbolic” __population and the number of jobs__ for the stations in the underground. You will also be given __the number of people that commute from one station to another__, through an OD matrix.

## read data

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import json
import re
from shapely.geometry import Point, LineString #this library is for manipulating geometric objects, and it is what geopandas uses to store geometries
from scipy.spatial import distance

import seaborn as sns
import folium
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
import numpy as np
from math import sqrt

In [2]:
# data = pd.read_csv("./WK10P/outputs/london_flows.csv", index_col=0)
Londonflow = pd.read_csv("./WK10P/outputs/london_flows.csv")
Londonflow = Londonflow.drop(Londonflow[Londonflow['station_destination'] == 'Battersea Park'].index)
Londonflow = Londonflow.drop(Londonflow[Londonflow['station_origin'] == 'Battersea Park'].index)
Londonflow.head()

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167


In [3]:
DTsubmat = pd.pivot_table(Londonflow, values ="flows", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
DTsubmat

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,599
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1224
Acton Town,NaN,NaN,NaN,3.0,17.0,NaN,35.0,0.0,NaN,11.0,...,77.0,3.0,6.0,9.0,NaN,0.0,NaN,0.0,NaN,3745
Aldgate,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,17.0,...,0.0,NaN,4.0,8.0,NaN,0.0,NaN,0.0,NaN,2886
Aldgate East,NaN,NaN,2.0,0.0,NaN,NaN,0.0,0.0,NaN,20.0,...,24.0,0.0,0.0,12.0,NaN,1.0,NaN,1.0,NaN,3172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.0,5.0,47.0,NaN,NaN,NaN,NaN,22.0,...,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4868
Woodgrange Park,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530
Woodside Park,NaN,NaN,1.0,26.0,11.0,NaN,0.0,NaN,NaN,59.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3093


# III. Models and calibration

## III.1. introduce the spatial interaction models

* Briefly introduce the spatial interaction models covered in the lectures using equations and defining the terms
* taking particular care in explaining the role of the parameters.

### UNCONSTRAINED MODEL

* ONLY THE BASIC constraint equation - 
total flows remain the same

### The Singly-Constrained Models

#### 2a: The Origin- Constrained Model

In the production-constrained model, $O_i$ does not have a parameter as it is a known constraint. $A_i$ is known as a <i>balancing factor</i> and is a vector of values which relate to each origin, $i$, which do the equivalent job to $k$ in the unconstrained/total constrained model but ensure that flow estimates from each origin sum to the known totals, $O_i$ rather than just the overall total.

#### 2b: The Destination-Constrained Model

### The Doubly Constrained Model


   

## III.2. calibrate the parameter for the cost function OF THE SELECTED MODEL

* Using the information of __population, jobs and flows__, 
* __select a spatial interaction model__ and 
* calibrate the __parameter__ for the cost function (usually denoted as __BETA__). It is essential that you justify the model selected.

### The Origin-Constrained Model

In [4]:
DTsubmat_III2 = pd.pivot_table(Londonflow, values ="flows", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
DTsubmat_III2

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,599
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1224
Acton Town,NaN,NaN,NaN,3.0,17.0,NaN,35.0,0.0,NaN,11.0,...,77.0,3.0,6.0,9.0,NaN,0.0,NaN,0.0,NaN,3745
Aldgate,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,17.0,...,0.0,NaN,4.0,8.0,NaN,0.0,NaN,0.0,NaN,2886
Aldgate East,NaN,NaN,2.0,0.0,NaN,NaN,0.0,0.0,NaN,20.0,...,24.0,0.0,0.0,12.0,NaN,1.0,NaN,1.0,NaN,3172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.0,5.0,47.0,NaN,NaN,NaN,NaN,22.0,...,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4868
Woodgrange Park,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530
Woodside Park,NaN,NaN,1.0,26.0,11.0,NaN,0.0,NaN,NaN,59.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3093


In [5]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE

In [6]:
#take the variables and produce logarithms of them
x_variables = ["jobs", "distance"]
log_x_vars = []
for x in x_variables:
    Londonflow[f"log_{x}"] = np.log(Londonflow[x]+1)
    log_x_vars.append(f"log_{x}")

#create the formula
formula = 'flows ~ station_origin + log_jobs + log_distance -1'

#run the regression
prodSim = smf.glm(formula = formula, 
                  data = Londonflow, 
                  family = sm.families.Poisson()).fit()
print(prodSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61431
Model:                            GLM   Df Residuals:                    61031
Model Family:                 Poisson   Df Model:                          399
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1903e+06
Date:                Mon, 01 May 2023   Deviance:                   2.2083e+06
Time:                        11:34:09   Pearson chi2:                 3.83e+06
No. Iterations:                    11   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

#### got gamma and beta

The $\gamma$ parameter related to the destination attractiveness: 0.7928

The $\beta$ distance decay parameter: 0.2795. Recall the negative sign in the equation.

```
No log

#create the formula (the "-1" indicates no intercept in the regression model).
formula = 'flows ~ station_origin + jobs + distance -1'

#run a production constrained sim
prodSim = smf.glm(formula = formula, data = Londonflow, family=sm.families.Poisson()).fit()

#let's have a look at it's summary
print(prodSim.summary())
```

### normal Model Estimates

In [7]:
Londonflow.head()

,station_origin,station_destination,flows,population,jobs,distance,log_jobs,log_distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271491,9.003627
1,Abbey Road,Beckton,1,599,442,8510.121774,6.093570,9.049129
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.501290,8.236539
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981438,8.534545
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.644004,7.709722


In [8]:
#create some Oi and Dj columns in the dataframe and store row and column totals in them:
#to create O_i, take cdatasub ...then... group by origcodenew ...then... summarise by calculating the sum of Total
O_i = pd.DataFrame(Londonflow.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
Londonflow = Londonflow.merge(O_i, on = "station_origin", how = "left" )

D_j = pd.DataFrame(Londonflow.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
Londonflow = Londonflow.merge(D_j, on = "station_destination", how = "left" )

In [9]:
#We can do this by pulling out the parameter values
coefs = pd.DataFrame(prodSim.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(station_origin)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "")
#then once you have done this you can join them back into the dataframes
Londonflow = Londonflow.merge(coefs, left_on="station_origin", right_on="coef", how = "left")
Londonflow.drop(columns = ["coef"], inplace = True)
#check this has worked
Londonflow.head()

/tmp/ipykernel_67/3470944051.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  coefs["coef"] = coefs["coef"].str.replace(x, "")


,station_origin,station_destination,flows,population,jobs,distance,log_jobs,log_distance,O_i,D_j,alpha_i
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271491,9.003627,599,78549,-1.677921
1,Abbey Road,Beckton,1,599,442,8510.121774,6.093570,9.049129,599,442,-1.677921
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.501290,8.236539,599,665,-1.677921
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981438,8.534545,599,58772,-1.677921
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.644004,7.709722,599,15428,-1.677921


#### get alpha, gamma, beta

In [10]:
alpha_i = prodSim.params[0:398]
gamma = prodSim.params[398]
beta = -prodSim.params[399]

In [11]:
alpha_i

station_origin[Abbey Road]         -1.677921
station_origin[Acton Central]      -0.723294
station_origin[Acton Town]         -1.282977
station_origin[Aldgate]            -1.822916
station_origin[Aldgate East]       -1.735682
                                      ...   
station_origin[Wood Street]        -0.493450
station_origin[Woodford]           -0.916078
station_origin[Woodgrange Park]    -0.427171
station_origin[Woodside Park]      -1.348228
station_origin[Woolwich Arsenal]    1.163911
Length: 398, dtype: float64

In [12]:
gamma

0.7927915378130382

In [13]:
beta

0.27948595814451616

In [14]:
Londonflow["est1"] = np.exp(Londonflow["alpha_i"]+ gamma*Londonflow["log_jobs"] - beta*Londonflow["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
Londonflow.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_jobs,log_distance,O_i,D_j,alpha_i,est1
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271491,9.003627,599,78549,-1.677921,114.621302
1,Abbey Road,Beckton,1,599,442,8510.121774,6.093570,9.049129,599,442,-1.677921,1.866201
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.501290,8.236539,599,665,-1.677921,3.235712
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981438,8.534545,599,58772,-1.677921,103.833030
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.644004,7.709722,599,15428,-1.677921,45.286285
5,Abbey Road,Crossharbour,1,599,1208,6686.475560,7.097549,8.807992,599,1208,-1.677921,4.424889
6,Abbey Road,Custom House,0,599,845,3824.855630,6.740519,8.249537,599,845,-1.677921,3.897266
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909,7.466799,9.048398,599,1748,-1.677921,5.544430
8,Abbey Road,Cyprus,7,599,850,6532.099618,6.746412,8.784637,599,850,-1.677921,3.371623
9,Abbey Road,Devons Road,1,599,611,3958.324171,6.416732,8.283829,599,611,-1.677921,2.986182


In [15]:
CalcRSquared(Londonflow["flows"], Londonflow["est1"])

0.21990903791445465

In [16]:
CalcRMSE(Londonflow["flows"], Londonflow["est1"])

120.402

### Assessing the model output

In [17]:
DTsubmat

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,599
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1224
Acton Town,NaN,NaN,NaN,3.0,17.0,NaN,35.0,0.0,NaN,11.0,...,77.0,3.0,6.0,9.0,NaN,0.0,NaN,0.0,NaN,3745
Aldgate,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,17.0,...,0.0,NaN,4.0,8.0,NaN,0.0,NaN,0.0,NaN,2886
Aldgate East,NaN,NaN,2.0,0.0,NaN,NaN,0.0,0.0,NaN,20.0,...,24.0,0.0,0.0,12.0,NaN,1.0,NaN,1.0,NaN,3172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.0,5.0,47.0,NaN,NaN,NaN,NaN,22.0,...,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4868
Woodgrange Park,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530
Woodside Park,NaN,NaN,1.0,26.0,11.0,NaN,0.0,NaN,NaN,59.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3093


In [18]:
#first round the estimates
Londonflow["est1"] = round(Londonflow["est1"],0)
#now we can create a pivot tabel to turn the paired list into a matrix, and compute the margins as well
DTsubmat_III2_2 = Londonflow.pivot_table(values ="est1", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
DTsubmat_III2_2

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,598.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1224.0
Acton Town,NaN,NaN,NaN,22.0,22.0,NaN,5.0,1.0,NaN,23.0,...,19.0,3.0,7.0,8.0,NaN,3.0,NaN,3.0,NaN,3742.0
Aldgate,NaN,NaN,5.0,NaN,26.0,NaN,NaN,1.0,NaN,21.0,...,11.0,NaN,5.0,4.0,NaN,2.0,NaN,2.0,NaN,2887.0
Aldgate East,NaN,NaN,5.0,28.0,NaN,NaN,2.0,1.0,NaN,22.0,...,11.0,2.0,5.0,4.0,NaN,2.0,NaN,2.0,NaN,3168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,10.0,33.0,34.0,NaN,NaN,NaN,NaN,32.0,...,22.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,4869.0
Woodgrange Park,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529.0
Woodside Park,NaN,NaN,7.0,21.0,21.0,NaN,3.0,NaN,NaN,22.0,...,15.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,3097.0


Here it is very easy to see the Origin Constrained working. 
* The sum across all destinations for each origin in the estimated matrix is exactly the same sum (give or take 1 or 2) across the observed matrics - $\sum_j T_{ij} = \sum_j \lambda_{ij} = O_i$, but clearly, 
* the same is not true when you sum across all origins for each destination - $\sum_i T_{ij} \neq \sum_i \lambda_{ij} \neq D_j$

In [19]:
Londonflow["est1"] = np.exp(Londonflow["alpha_i"]+ gamma*Londonflow["log_jobs"] - beta*Londonflow["log_distance"])
Londonflow["est1"] = round(Londonflow["est1"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
Londonflow.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_jobs,log_distance,O_i,D_j,alpha_i,est1
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271491,9.003627,599,78549,-1.677921,115.0
1,Abbey Road,Beckton,1,599,442,8510.121774,6.093570,9.049129,599,442,-1.677921,2.0
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.501290,8.236539,599,665,-1.677921,3.0
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981438,8.534545,599,58772,-1.677921,104.0
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.644004,7.709722,599,15428,-1.677921,45.0
5,Abbey Road,Crossharbour,1,599,1208,6686.475560,7.097549,8.807992,599,1208,-1.677921,4.0
6,Abbey Road,Custom House,0,599,845,3824.855630,6.740519,8.249537,599,845,-1.677921,4.0
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909,7.466799,9.048398,599,1748,-1.677921,6.0
8,Abbey Road,Cyprus,7,599,850,6532.099618,6.746412,8.784637,599,850,-1.677921,3.0
9,Abbey Road,Devons Road,1,599,611,3958.324171,6.416732,8.283829,599,611,-1.677921,3.0


### try negative expotential 

In [20]:
#create the formula
NE_formula = 'flows ~ station_origin + jobs + distance -1'

#run the regression
NexpoSim = smf.glm(formula = NE_formula, 
                  data = Londonflow, 
                  family = sm.families.Poisson()).fit()
print(NexpoSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61431
Model:                            GLM   Df Residuals:                    61031
Model Family:                 Poisson   Df Model:                          399
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1253e+06
Date:                Mon, 01 May 2023   Deviance:                   2.0784e+06
Time:                        11:34:27   Pearson chi2:                 3.08e+06
No. Iterations:                     8   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

#### get alpha gamma beta

In [21]:
alpha_i_ne = NexpoSim.params[0:398]
gamma_ne = NexpoSim.params[398]
beta_ne = -NexpoSim.params[399]

In [22]:
beta_ne

0.00014377929133584962

In [23]:
gamma_ne

4.151346142465418e-05

In [24]:
alpha_i_ne

station_origin[Abbey Road]          2.753050
station_origin[Acton Central]       4.427951
station_origin[Acton Town]          4.263440
station_origin[Aldgate]             3.128385
station_origin[Aldgate East]        3.204054
                                      ...   
station_origin[Wood Street]         4.609125
station_origin[Woodford]            5.068470
station_origin[Woodgrange Park]     4.457912
station_origin[Woodside Park]       4.771284
station_origin[Woolwich Arsenal]    6.030280
Length: 398, dtype: float64

In [25]:
#We can do this by pulling out the parameter values
coefs = pd.DataFrame(prodSim.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i_ne", "index":"coef"}, inplace = True)
to_repl = ["(station_origin)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "")
#then once you have done this you can join them back into the dataframes
Londonflow = Londonflow.merge(coefs, left_on="station_origin", right_on="coef", how = "left")
Londonflow.drop(columns = ["coef"], inplace = True)
#check this has worked
Londonflow.head()

/tmp/ipykernel_67/2191618147.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  coefs["coef"] = coefs["coef"].str.replace(x, "")


,station_origin,station_destination,flows,population,jobs,distance,log_jobs,log_distance,O_i,D_j,alpha_i,est1,alpha_i_ne
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271491,9.003627,599,78549,-1.677921,115.0,-1.677921
1,Abbey Road,Beckton,1,599,442,8510.121774,6.093570,9.049129,599,442,-1.677921,2.0,-1.677921
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.501290,8.236539,599,665,-1.677921,3.0,-1.677921
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981438,8.534545,599,58772,-1.677921,104.0,-1.677921
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.644004,7.709722,599,15428,-1.677921,45.0,-1.677921


In [26]:
Londonflow["est1_ne"] = np.exp(Londonflow["alpha_i_ne"]+ gamma_ne*Londonflow["log_jobs"] - beta_ne*Londonflow["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
Londonflow.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_jobs,log_distance,O_i,D_j,alpha_i,est1,alpha_i_ne,est1_ne
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271491,9.003627,599,78549,-1.677921,115.0,-1.677921,0.186607
1,Abbey Road,Beckton,1,599,442,8510.121774,6.093570,9.049129,599,442,-1.677921,2.0,-1.677921,0.186566
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.501290,8.236539,599,665,-1.677921,3.0,-1.677921,0.186591
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981438,8.534545,599,58772,-1.677921,104.0,-1.677921,0.186618
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.644004,7.709722,599,15428,-1.677921,45.0,-1.677921,0.186630
5,Abbey Road,Crossharbour,1,599,1208,6686.475560,7.097549,8.807992,599,1208,-1.677921,4.0,-1.677921,0.186580
6,Abbey Road,Custom House,0,599,845,3824.855630,6.740519,8.249537,599,845,-1.677921,4.0,-1.677921,0.186593
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909,7.466799,9.048398,599,1748,-1.677921,6.0,-1.677921,0.186577
8,Abbey Road,Cyprus,7,599,850,6532.099618,6.746412,8.784637,599,850,-1.677921,3.0,-1.677921,0.186578
9,Abbey Road,Devons Road,1,599,611,3958.324171,6.416732,8.283829,599,611,-1.677921,3.0,-1.677921,0.186589


In [27]:
CalcRSquared(Londonflow["flows"], Londonflow["est1_ne"])

0.05311175952232571

# IV. Scenarios
## IV.1. Scenario A: 
* assume that __Canary Wharf__ has a __50% decrease in jobs__ after Brexit. 
* Using the __calibrated parameter BETA__, __compute the new flows__ for scenario A. 
* Make sure the number of commuters is conserved, and 
* explain how you ensured this.

In [28]:
flow_scenarioA = Londonflow
flow_scenarioB = Londonflow
flow_scenarioB1 = Londonflow

In [29]:
def new_sal(row):
    if row["station_destination"] == "Canary Wharf":
        val = row["jobs"]*0.5
    else:
        val = row["jobs"]
    return val
        
flow_scenarioA["ScenarioA_jobs"] = flow_scenarioA.apply(new_sal, axis =1)
flow_scenarioA.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_jobs,log_distance,O_i,D_j,alpha_i,est1,alpha_i_ne,est1_ne,ScenarioA_jobs
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,11.271491,9.003627,599,78549,-1.677921,115.0,-1.677921,0.186607,78549.0
1,Abbey Road,Beckton,1,599,442,8510.121774,6.093570,9.049129,599,442,-1.677921,2.0,-1.677921,0.186566,442.0
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.501290,8.236539,599,665,-1.677921,3.0,-1.677921,0.186591,665.0
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981438,8.534545,599,58772,-1.677921,104.0,-1.677921,0.186618,29386.0
4,Abbey Road,Canning Town,37,599,15428,2228.923167,9.644004,7.709722,599,15428,-1.677921,45.0,-1.677921,0.186630,15428.0
5,Abbey Road,Crossharbour,1,599,1208,6686.475560,7.097549,8.807992,599,1208,-1.677921,4.0,-1.677921,0.186580,1208.0
6,Abbey Road,Custom House,0,599,845,3824.855630,6.740519,8.249537,599,845,-1.677921,4.0,-1.677921,0.186593,845.0
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909,7.466799,9.048398,599,1748,-1.677921,6.0,-1.677921,0.186577,1748.0
8,Abbey Road,Cyprus,7,599,850,6532.099618,6.746412,8.784637,599,850,-1.677921,3.0,-1.677921,0.186578,850.0
9,Abbey Road,Devons Road,1,599,611,3958.324171,6.416732,8.283829,599,611,-1.677921,3.0,-1.677921,0.186589,611.0


### plug these new values into the model

see how this changes the flows in the system

In [30]:
beta

0.27948595814451616


``` 
Londonflow["est2"] = np.exp(Londonflow["alpha_i"] + gamma*np.log(Londonflow["ScenarioA_jobs"]) - beta*Londonflow["log_distance"])

Londonflow["est2"] = round(Londonflow["est2"],0)
#now we can convert the pivot table into a matrix
DTsubmat_IV1A = Londonflow.pivot_table(values ="est2", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
DTsubmat_IV1A.iloc[:, 50:60]
```

by increasing the average salary in Barking and Dagenham, we’ve increased flows into Barking and Dagenham, but have not reduced the flows into other zones - the original constraints are still working on the other zones. 

One way to get around this, now that we have calibrated our parameters, is to return to the multiplicative model in Equation 1 and run this model after calculating our own  𝐴𝑖  balancing factors.

In [31]:
#calculate some new Dj^gamma and d_ij^beta values
A_jobs_gamma = flow_scenarioA["jobs"]**gamma
dist_beta = flow_scenarioA["distance"]**-beta
#calcualte the first stage of the Ai values
flow_scenarioA["Ai1"] = A_jobs_gamma * dist_beta
#now do the sum over all js bit
A_i = pd.DataFrame(flow_scenarioA.groupby(["station_origin"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i"}, inplace=True)
#and write the A_i values back into the dataframe
flow_scenarioA = flow_scenarioA.merge(A_i, left_on="station_origin", right_index=True, how="left")

You should see that your new estimates are exactly the same as your first estimates. If they’re not, then something has gone wrong. 

In [32]:
#to check everything works, recreate the original estimates
flow_scenarioA["est3"] = flow_scenarioA["A_i"]*flow_scenarioA["O_i"]*A_jobs_gamma*dist_beta
#round
flow_scenarioA["est3"] = round(flow_scenarioA["est3"])
#check
flow_scenarioA[["est1", "est3"]]

,est1,est3
0,115.0,115.0
1,2.0,2.0
2,3.0,3.0
3,104.0,104.0
4,45.0,45.0
...,...,...
61426,140.0,140.0
61427,234.0,234.0
61428,29.0,29.0
61429,64.0,64.0


In [33]:
#calculate some new Dj^gamma and d_ij^beta values
Dj3_gamma = flow_scenarioA["ScenarioA_jobs"]**gamma
#calcualte the first stage of the Ai values
flow_scenarioA["Ai1"] = Dj3_gamma * dist_beta
#now do the sum over all js bit
A_i = pd.DataFrame(flow_scenarioA.groupby(["station_origin"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i2"}, inplace=True)
#and write the A_i values back into the dataframe
flow_scenarioA = flow_scenarioA.merge(A_i, left_on="station_origin", right_index=True, how="left")

In [34]:
#to check everything works, recreate the original estimates
flow_scenarioA["est4"] = flow_scenarioA["A_i2"]*flow_scenarioA["O_i"]*Dj3_gamma*dist_beta
#round
flow_scenarioA["est4"] = round(flow_scenarioA["est4"])

DTsubmat_IV1A2 = flow_scenarioA.pivot_table(values ="est4", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
DTsubmat_IV1A2.iloc[:, 50:60]

station_destination,Cambridge Heath,Camden Road,Camden Town,Canada Water,Canary Wharf,Canning Town,Cannon Street,Canonbury,Canons Park,Carpenders Park
station_origin,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,65.0,49.0,NaN,NaN,NaN,NaN
Acton Central,NaN,24.0,NaN,80.0,NaN,NaN,NaN,7.0,NaN,3.0
Acton Town,NaN,NaN,20.0,48.0,61.0,35.0,15.0,NaN,2.0,NaN
Aldgate,NaN,NaN,15.0,39.0,49.0,27.0,16.0,NaN,1.0,NaN
Aldgate East,NaN,NaN,16.0,47.0,57.0,31.0,17.0,NaN,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,26.0,71.0,98.0,62.0,22.0,NaN,NaN,NaN
Woodgrange Park,NaN,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Woodside Park,NaN,NaN,20.0,43.0,56.0,33.0,14.0,NaN,NaN,NaN


In [35]:
DTsubmat

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,599
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1224
Acton Town,NaN,NaN,NaN,3.0,17.0,NaN,35.0,0.0,NaN,11.0,...,77.0,3.0,6.0,9.0,NaN,0.0,NaN,0.0,NaN,3745
Aldgate,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,17.0,...,0.0,NaN,4.0,8.0,NaN,0.0,NaN,0.0,NaN,2886
Aldgate East,NaN,NaN,2.0,0.0,NaN,NaN,0.0,0.0,NaN,20.0,...,24.0,0.0,0.0,12.0,NaN,1.0,NaN,1.0,NaN,3172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.0,5.0,47.0,NaN,NaN,NaN,NaN,22.0,...,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4868
Woodgrange Park,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530
Woodside Park,NaN,NaN,1.0,26.0,11.0,NaN,0.0,NaN,NaN,59.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3093


In [36]:
DTsubmat_IV1A2

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,599.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1224.0
Acton Town,NaN,NaN,NaN,22.0,23.0,NaN,5.0,1.0,NaN,24.0,...,19.0,3.0,7.0,8.0,NaN,3.0,NaN,3.0,NaN,3743.0
Aldgate,NaN,NaN,5.0,NaN,26.0,NaN,NaN,1.0,NaN,21.0,...,11.0,NaN,5.0,4.0,NaN,2.0,NaN,2.0,NaN,2882.0
Aldgate East,NaN,NaN,5.0,28.0,NaN,NaN,2.0,1.0,NaN,22.0,...,12.0,2.0,6.0,4.0,NaN,2.0,NaN,2.0,NaN,3171.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,10.0,33.0,35.0,NaN,NaN,NaN,NaN,33.0,...,23.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,4871.0
Woodgrange Park,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529.0
Woodside Park,NaN,NaN,7.0,21.0,21.0,NaN,3.0,NaN,NaN,23.0,...,15.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,3093.0


In [37]:
DTsubmat_IV1A2_all = DTsubmat_IV1A2.tail(1)
transposed_IV1A2_all = DTsubmat_IV1A2_all.T
transposed_IV1A2_all.rename(columns={'All': 'All_half'}, inplace=True)
transposed_IV1A2_all

station_origin,All_half
station_destination,
Abbey Road,241.0
Acton Central,429.0
Acton Town,2018.0
Aldgate,6063.0
Aldgate East,6638.0
...,...
Woodford,696.0
Woodgrange Park,121.0
Woodside Park,655.0


In [38]:
DTsubmat_all = DTsubmat.tail(1)
transposed_DT_all = DTsubmat_all.T
transposed_DT_all

station_origin,All
station_destination,
Abbey Road,345.0
Acton Central,750.0
Acton Town,2202.0
Aldgate,7782.0
Aldgate East,7932.0
...,...
Woodford,706.0
Woodgrange Park,242.0
Woodside Park,745.0


In [39]:
merged_DT_IVA = pd.merge(transposed_DT_all, transposed_IV1A2_all, on='station_destination')
merged_DT_IVA['changes'] = merged_DT_IVA['All_half'] - merged_DT_IVA['All']
merged_DT_IVA.to_csv('merged_DT_IVA.csv', index=True)
merged_DT_IVA = pd.read_csv("./merged_DT_IVA.csv")
merged_DT_IVA = merged_DT_IVA.sort_values(by='changes', ascending=False)
merged_DT_IVA.head(10)

,station_destination,All,All_half,changes
326,Stratford,55954.0,66544.0,10590.0
386,Whitechapel,17633.0,19099.0,1466.0
371,West Brompton,5859.0,7123.0,1264.0
388,Willesden Junction,4165.0,5248.0,1083.0
374,West Ham,5487.0,6456.0,969.0
375,West Hampstead,5856.0,6526.0,670.0
26,Bethnal Green,4660.0,5290.0,630.0
141,Gunnersbury,4775.0,5388.0,613.0
192,Kew Gardens,792.0,1165.0,373.0
53,Canada Water,20443.0,20782.0,339.0


In [40]:
merged_DT_IVA.tail(10)

,station_destination,All,All_half,changes
227,Moorgate,24574.0,17193.0,-7381.0
119,Farringdon,25592.0,17512.0,-8080.0
138,Green Park,26754.0,18258.0,-8496.0
213,London Bridge,29930.0,20151.0,-9779.0
355,Victoria,33251.0,21560.0,-11691.0
197,King's Cross St. Pancras,33330.0,21221.0,-12109.0
251,Oxford Circus,44368.0,27511.0,-16857.0
15,Bank and Monument,78549.0,54715.0,-23834.0
54,Canary Wharf,58772.0,26260.0,-32512.0
398,All,1542391.0,1229322.0,-313069.0


There are a number of things to note here. 

__Firstly, flows into Canary Wharf have virtually decreased, overall flows also reduced.__

*Secondly, Barking and Dagenham was a poor estimate anyway - the model was very much over estimating flows into this Borough. Increasing the salary into this borough has significantly increased flows, so this indicates that there are probably lots of other things that might be discouraging people from working in this borough.*

__Thirdly, Our origin constraints are now holding again.__

## IV.2. Scenario B: 
* assume that there is a significant increase in the cost of transport. 
* Select 2 values for the parameter in the cost function reflecting scenario B. 
* Recompute the distribution of flows.

### if beta = 1 gamma = 0.27

In [41]:
#calculate some new Dj^gamma and d_ij^beta values
beta = 0.5
A_jobs_gamma = flow_scenarioB["jobs"]**gamma
dist_beta_B5 = flow_scenarioB["distance"]**-beta
#calcualte the first stage of the Ai values
flow_scenarioB["Ai1_B5"] = A_jobs_gamma * dist_beta_B5
#now do the sum over all js bit
A_i_B5 = pd.DataFrame(flow_scenarioB.groupby(["station_origin"])["Ai1_B5"].agg(np.sum))
#now divide into 1
A_i_B5["Ai1_B5"] = 1/A_i_B5["Ai1_B5"]
A_i_B5.rename(columns={"Ai1_B5":"A_i_B5"}, inplace=True)
#and write the A_i values back into the dataframe
flow_scenarioB = flow_scenarioB.merge(A_i_B5, left_on="station_origin", right_index=True, how="left")

In [42]:
#to check everything works, recreate the original estimates
flow_scenarioB["est_B5"] = flow_scenarioB["A_i_B5"]*Londonflow["O_i"]*A_jobs_gamma*dist_beta_B5
#round
flow_scenarioB["est_B5"] = round(flow_scenarioB["est_B5"])

DTsubmat_IV1B5 = flow_scenarioB.pivot_table(values ="est_B5", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
DTsubmat_IV1B5

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,598.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1221.0
Acton Town,NaN,NaN,NaN,21.0,21.0,NaN,6.0,1.0,NaN,22.0,...,18.0,3.0,6.0,10.0,NaN,2.0,NaN,3.0,NaN,3751.0
Aldgate,NaN,NaN,4.0,NaN,33.0,NaN,NaN,0.0,NaN,22.0,...,8.0,NaN,4.0,3.0,NaN,1.0,NaN,1.0,NaN,2891.0
Aldgate East,NaN,NaN,4.0,36.0,NaN,NaN,1.0,0.0,NaN,23.0,...,8.0,1.0,4.0,3.0,NaN,2.0,NaN,2.0,NaN,3179.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,9.0,34.0,36.0,NaN,NaN,NaN,NaN,33.0,...,19.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,4865.0
Woodgrange Park,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,532.0
Woodside Park,NaN,NaN,6.0,21.0,21.0,NaN,2.0,NaN,NaN,23.0,...,14.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,3092.0


In [43]:
DTsubmat_IV1B1_all = DTsubmat_IV1B5.tail(1)
transposed_IV1B1_all = DTsubmat_IV1B1_all.T
transposed_IV1B1_all.rename(columns={'All': 'All_beta5'}, inplace=True)
transposed_IV1B1_all

station_origin,All_beta5
station_destination,
Abbey Road,269.0
Acton Central,398.0
Acton Town,1866.0
Aldgate,6128.0
Aldgate East,6753.0
...,...
Woodford,607.0
Woodgrange Park,125.0
Woodside Park,568.0


In [44]:
DTsubmat_all = DTsubmat.tail(1)
transposed_DT_all = DTsubmat_all.T
transposed_DT_all

station_origin,All
station_destination,
Abbey Road,345.0
Acton Central,750.0
Acton Town,2202.0
Aldgate,7782.0
Aldgate East,7932.0
...,...
Woodford,706.0
Woodgrange Park,242.0
Woodside Park,745.0


In [45]:
merged_DT_IVB = pd.merge(transposed_DT_all, transposed_IV1B1_all, on='station_destination')
merged_DT_IVB['changes'] = merged_DT_IVB['All_beta5'] - merged_DT_IVB['All']
merged_DT_IVB.to_csv('merged_DT_IVB.csv', index=True)
merged_DT_IVB

station_origin,All,All_beta5,changes
station_destination,,,
Abbey Road,345.0,269.0,-76.0
Acton Central,750.0,398.0,-352.0
Acton Town,2202.0,1866.0,-336.0
Aldgate,7782.0,6128.0,-1654.0
Aldgate East,7932.0,6753.0,-1179.0
...,...,...,...
Woodford,706.0,607.0,-99.0
Woodgrange Park,242.0,125.0,-117.0
Woodside Park,745.0,568.0,-177.0


In [46]:
merged_DT_IVB = pd.read_csv("./merged_DT_IVB.csv")
merged_DT_IVB = merged_DT_IVB.sort_values(by='changes', ascending=False)
merged_DT_IVB.head(10)

,station_destination,All,All_beta5,changes
326,Stratford,55954.0,64154.0,8200.0
386,Whitechapel,17633.0,19244.0,1611.0
371,West Brompton,5859.0,6944.0,1085.0
374,West Ham,5487.0,6352.0,865.0
53,Canada Water,20443.0,21197.0,754.0
388,Willesden Junction,4165.0,4869.0,704.0
26,Bethnal Green,4660.0,5289.0,629.0
375,West Hampstead,5856.0,6429.0,573.0
201,Lambeth North,1319.0,1596.0,277.0
322,Stepney Green,1879.0,2121.0,242.0


In [47]:
merged_DT_IVB.tail(10)

,station_destination,All,All_beta5,changes
227,Moorgate,24574.0,17772.0,-6802.0
119,Farringdon,25592.0,17752.0,-7840.0
138,Green Park,26754.0,18552.0,-8202.0
213,London Bridge,29930.0,20661.0,-9269.0
355,Victoria,33251.0,21805.0,-11446.0
197,King's Cross St. Pancras,33330.0,21615.0,-11715.0
54,Canary Wharf,58772.0,44384.0,-14388.0
251,Oxford Circus,44368.0,28169.0,-16199.0
15,Bank and Monument,78549.0,53105.0,-25444.0
398,All,1542391.0,1229205.0,-313186.0


### if beta = 0.5 gamma = 0.27

#calculate some new Dj^gamma and d_ij^beta values
beta = 0.8
A_jobs_gamma = flow_scenarioB1["jobs"]**gamma
dist_beta_B1 = flow_scenarioB1["distance"]**-beta
#calcualte the first stage of the Ai values
flow_scenarioB1["Ai1_B1"] = A_jobs_gamma * dist_beta_B1
#now do the sum over all js bit
A_i_B1 = pd.DataFrame(flow_scenarioB1.groupby(["station_origin"])["Ai1_B1"].agg(np.sum))
#now divide into 1
A_i_B1["Ai1_B1"] = 1/A_i_B1["Ai1_B1"]
A_i_B1.rename(columns={"Ai1_B1":"A_i_B1"}, inplace=True)
#and write the A_i values back into the dataframe
flow_scenarioB1 = flow_scenarioB1.merge(A_i_B1, left_on="station_origin", right_index=True, how="left")

#to check everything works, recreate the original estimates
flow_scenarioB1["est_B1"] = flow_scenarioB1["A_i_B1"]*flow_scenarioB1["O_i"]*A_jobs_gamma*dist_beta_B5
#round
flow_scenarioB1["est_B1"] = round(flow_scenarioB1["est_B1"])

DTsubmat_IV2B1 = flow_scenarioB1.pivot_table(values ="est_B1", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
DTsubmat_IV2B1

## IV.3. 

* Discuss how the flows change for the 3 different situations: 
__scenario A__, and 
__scenario B__ with __two selections of parameters__. 

* __Which scenario__ would have more impact in the redistribution of flows? 
* Explain and __justify your answers__ using the results of the analysis.

Where have flows from dropped?

Where have flows increased from?

Which flow was most affected?

Why do you think that is?

Is the production constrained model appropriate?

Would jobs respond to the increase in households, or would it be the other way round?